In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
import sklearn.model_selection as model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import pickle
import xgboost as xgb
from sklearn.metrics import confusion_matrix
import pickle

from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, precision_score
from sklearn.neural_network import MLPClassifier
from sklearn import svm, tree
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from tqdm import tqdm #creates progress bar to let you know how long is left till function is complete
import xgboost as xgb
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTENC
from imblearn.pipeline import Pipeline
from pandas.tseries.offsets import DateOffset

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv('/content/drive/MyDrive/DoR/cases_train_processed.csv')
df = df.drop(['Unnamed: 0','index','source','additional_information','Last_Update','Lat_right','Long_right','Province_State','Country_Region','dist_between_in_km'],1)
df['Confirmed'].fillna(df['Confirmed'].mean(),inplace=True)
df['Deaths'].fillna(df.Deaths.mean(),inplace=True)
df['Recovered'].fillna(df.Recovered.mean(),inplace=True)
df['Active'].fillna(df.Active.mean(),inplace=True)
df['Incidence_Rate'].fillna(df.Incidence_Rate.mean(),inplace=True)
df['Case-Fatality_Ratio'].fillna(df['Case-Fatality_Ratio'].mean(),inplace=True)
df.date_confirmation = pd.to_datetime(df.date_confirmation,infer_datetime_format=True) 
df.Combined_Key.fillna((df.province+" ,"+df.country),inplace=True)

In [258]:
# df.age = df.age.astype('object')
test_df = df.loc[0:20].copy()


IntervalIndex([(2020-01-02, 2020-01-16], (2020-01-16, 2020-01-30], (2020-01-30, 2020-02-13], (2020-02-13, 2020-02-27], (2020-02-27, 2020-03-12] ... (2020-10-08, 2020-10-22], (2020-10-22, 2020-11-05], (2020-11-05, 2020-11-19], (2020-11-19, 2020-12-03], (2020-12-03, 2020-12-17]],
              closed='right',
              dtype='interval[datetime64[ns]]')

# Make 2 week bins for date_confirmation 

In [4]:
def getDateBins(data):

  """ funtion takes in the date column, creates an array of 2 week intervals
   starting from the earliest day in the dataset
    and returns a dictionary of ordinal data where the k is the new label
    and value is the 2 week interval.
    ex: earliest confrimed date is 2020-01-02 -> 
    interval is (2020-01-02, 2020-01-16). dict returns 1:(2020-01-02, 2020-01-16)
  """
  start_date = data.min() - DateOffset(days=14)#earliest date + buffer
  print("start date is ",start_date)
  end_date = data.max() + DateOffset(days=14) #buffer to make sure nothing missed
  print("end date is ", end_date)
  date_interval = pd.interval_range(start=start_date,end=end_date,freq='14D')
  date_dict = {}
  for i in range(len(date_interval)):
    date_dict[i+1] = date_interval[i]
  return date_dict

def binDate(row_date,ddict):
  """ input is date_confrimation column. if date is in interval
  returns the dictionary key"""
  for k,v in ddict.items():
    # if row_date in v:
    #   print("here is key ",k)
    #   print("here is interval",v)
    if row_date in v:
      return k


In [5]:
#add bins to dataframe
%%time
dateBinDict = getDateBins(df.date_confirmation)
df['date_labels'] = df.date_confirmation.apply(lambda curr_date : binDate(curr_date,dateBinDict))

start date is  2019-12-19 00:00:00
end date is  2020-12-20 00:00:00
CPU times: user 4 s, sys: 37.6 ms, total: 4.04 s
Wall time: 4.06 s


# TEsting Over and Under Sampling 
----------------------------------------------------------

In [7]:
#drops duplicated columns created
def dropDuplicates(data):
    duplicates = data.columns[data.columns.duplicated()]
    if len(duplicates) > 0:
        data = data.loc[:,~data.columns.duplicated()]
    return data

#drops all the columns not used in X_train
def colsToDrop(dataframe):
    if 'outcome' in dataframe.columns:
        dataframe = dataframe.drop('outcome',1)
        print("dropping the outcome column")
    if 'Combined_Key' in dataframe.columns:
        dataframe = dataframe.drop('Combined_Key',1)
        print("dropping Combined_Key")
    if 'dist_between_in_km' in dataframe.columns:
        dataframe = dataframe.drop('dist_between_in_km',1)
        print("dropping dist in km column")
    if 'date_confirmation' in dataframe.columns:
        dataframe = dataframe.drop('date_confirmation',1)
        print("dropping date")
    if 'date_labels' in dataframe.columns:
        dataframe.date_labels = dataframe.date_labels.astype('object')
        print("converting date to categorical")
    if 'age' in dataframe.columns:
        dataframe.age = dataframe.age.astype('object')
        print("converting age to categorical")
    return dataframe
    
    # once hot encode and add new cols to dataframe
def oneHotEncode_df(dataframe):
    dataframe = colsToDrop(dataframe)
    col2Encode = list(dataframe.select_dtypes(include=['object'])) #gets a list of all the features that are objects assumption is that those are categorical
    dummies = pd.get_dummies(dataframe,columns=col2Encode,prefix=col2Encode,sparse=True)
    res = pd.concat([dataframe, dummies], axis=1)
    #if we decide to drop one hot encoded values
    res = res.drop(col2Encode, axis=1)
    output = dropDuplicates(res)
    return output

    # SMOTENC needs a list of the indices of all the categorical variables in dataset
    # ex: if country is column 2 and age is col 13. returns [2,13]
def getCategoricalIndices(dataframe):
    #get the indices of all the categorical variables
    numericalVar = list(dataframe.select_dtypes(include=['float64']))
    catlist = list(dataframe.columns.difference(numericalVar))
    print("these are the categorical features: {}".format(catlist))
    indlist = []
    for i in catlist:
        indlist.append(dataframe.columns.get_loc(i))
    return indlist

In [8]:
%%time
# Perform Oversampling before ohe.
#so drop columns not used in independent var 
sparse_df = colsToDrop(df)
#get list of categorical indices
catIndList = getCategoricalIndices(sparse_df)

dropping the outcome column
dropping Combined_Key
dropping date
converting date to categorical
converting age to categorical
these are the categorical features: ['age', 'country', 'date_labels', 'province', 'sex']
CPU times: user 109 ms, sys: 1.9 ms, total: 111 ms
Wall time: 117 ms


In [420]:
sparse_df.date_labels.value_counts()

12    111994
11     72764
10     30956
9      25954
8      17054
7      16305
2      15690
21      8962
15      8941
23      8707
6       8695
26      8619
19      8246
4       8204
17      7714
13      7122
3        742
5        441
25       323
14        76
24        65
1         60
Name: date_labels, dtype: int64

In [401]:
%%time
#oversample 
oversample = SMOTENC(categorical_features=catIndList,random_state=0,sampling_strategy='not majority')
x_o,y_o = oversample.fit_resample(sparse_df,df.outcome)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


CPU times: user 25min 41s, sys: 8.35 s, total: 25min 49s
Wall time: 25min 38s


In [402]:
# join dependent and independent var back together to save to csv
over_np = np.column_stack([x_o,y_o])

In [403]:
#creating headers to convert oversample data back to dataframe b4 saving to csv
coll = list(sparse_df.columns)
coll.append('outcome')
coll

['age',
 'sex',
 'province',
 'country',
 'Confirmed',
 'Deaths',
 'Recovered',
 'Active',
 'Incidence_Rate',
 'Case-Fatality_Ratio',
 'date_labels',
 'outcome']

In [424]:
# newO_df = pd.DataFrame(over_np,columns=coll)
newO_df

,age,sex,province,country,Confirmed,Deaths,Recovered,Active,Incidence_Rate,Case-Fatality_Ratio,date_labels,outcome
0,7,Unknown,Delhi,India,238828,4907,201671,32250,1276.41,2.05462,12,recovered
1,7,Unknown,Uttar Pradesh,India,342788,4869,270094,67825,144.1,1.42041,11,hospitalized
2,7,Unknown,Maharashtra,India,2859,2,2639,218,464.331,0.0699545,12,hospitalized
3,3,female,Baden-Wurttemberg,Germany,46779,1871,41228,3680,422.592,3.99966,7,nonhospitalized
4,7,Unknown,Gujarat,India,120336,3286,100974,16076,188.401,2.73069,11,hospitalized
...,...,...,...,...,...,...,...,...,...,...,...,...
599987,7,Unknown,Bihar,India,165218,859,151750,12609,132.386,0.519919,12,recovered
599988,7,Unknown,Uttar Pradesh,India,238828,4907,201671,32250,1276.41,2.05462,10,recovered
599989,7,Unknown,Maharashtra,India,2859,2,2639,218,464.331,0.0699545,11,recovered
599990,7,female,Gujarat,India,120336,3286,100974,16076,188.401,2.73069,12,recovered


In [404]:
# save oversampled data to csv
pd.DataFrame(over_np,columns=coll).to_csv("/content/drive/MyDrive/DoR/oversampledWithDates.csv")

------------------------------ END OF OVERSAMPLING----------------------------- 

**------------------------------ START OF ANALYSIS WITH OVERSAMPLED DATA--------------------------------------------------------------------------**

In [9]:
o_df = pd.read_csv('/content/drive/MyDrive/DoR/oversampledWithDates.csv')

In [10]:
# ensure age is categorical, drop unwanted columns
o_df.age = o_df.age.astype('object')
y_tr = o_df.outcome
X_tr = o_df
if 'outcome' in X_tr.columns:
      X_tr = X_tr.drop('outcome',1)
      print("dropping the outcome column")
if 'Unnamed: 0' in X_tr.columns:
      X_tr = X_tr.drop('Unnamed: 0',1)
      print("dropping the Unnamed column")      



dropping the outcome column
dropping the Unnamed column


--------------------------------------------------------

# Adding more deceased to original dataset. Target 5% and 10%

In [35]:
deceasedSubset = o_df[o_df.outcome=='deceased'].sample(n=40000) #10%
deceasedSubset = deceasedSubset.drop('Unnamed: 0',1)
deceasedSubset.columns

Index(['age', 'sex', 'province', 'country', 'Confirmed', 'Deaths', 'Recovered',
       'Active', 'Incidence_Rate', 'Case-Fatality_Ratio', 'date_labels',
       'outcome'],
      dtype='object')

In [36]:
deceased_df = sparse_df.copy()
deceased_df['outcome'] = df.outcome.copy()


In [37]:
#proportions before adding more to dataset
deceased_df.outcome.value_counts() /len(deceased_df.outcome)

nonhospitalized    0.408009
hospitalized       0.340012
recovered          0.239741
deceased           0.012238
Name: outcome, dtype: float64

In [38]:
#proportions after adding more to dataset
deceased_df = deceased_df.append(deceasedSubset)
deceased_df.outcome.value_counts() /len(deceased_df.outcome)

nonhospitalized    0.367972
hospitalized       0.306648
recovered          0.216216
deceased           0.109164
Name: outcome, dtype: float64

In [39]:
#save to csv for later analysis
deceased_df.to_csv('/content/drive/MyDrive/DoR/deceased10percentwithDate.csv')

In [41]:
y_tr = deceased_df.outcome
X_tr = deceased_df.drop('outcome',1)

In [42]:
%%time
# add dummy variables  
ohe_df = oneHotEncode_df(X_tr)
ohe_df
res = ohe_df.to_numpy()

converting date to categorical
converting age to categorical
CPU times: user 2.65 s, sys: 2.52 s, total: 5.16 s
Wall time: 5.18 s


## ---------------------------- END OF PROCESSING AND OHE --------------------------------------

In [ ]:
%%time
# #transform data
# oversample = SMOTENC(categorical_features=indlist,random_state=0,sampling_strategy='not majority')
# undersample = RandomUnderSampler()
# rfclassifier = RandomForestClassifier(n_estimators=25,random_state=0)
# #define pipeline
# steps = [('over',oversample),('under',undersample),('model',rfclassifier)]
# pipeline = Pipeline(steps=steps)
# #eval pipeline
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2,random_state=0)
# scores = cross_val_score(pipeline,x,y,scoring='f1',cv=cv,n_jobs=1)
# print('Mean F1: %.3f' %mean(scores))


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [118]:
%%time
# X = res
# y = df['outcome'].values
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
xgb_model = xgb.XGBClassifier(n_estimators=50)
xgb_model.fit(X_train,y_train)
pickle.dump( xgb_model, open( "../models/xgboost50.pkl", "wb" ) )

C:\Users\Oliver Fujiki\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:17:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [ ]:
trueXGBoost = pickle.load( open( '../models/xgboost50.pkl', "rb" ) )
XGprediction = list(trueXGBoost.predict(X_test))
xgConfusionMatrix = confusion_matrix(y_test, XGprediction, labels=["recovered","nonhospitalized","hospitalized","deceased"])

In [ ]:
pd.DataFrame(xgConfusionMatrix).rename(index = {0:"recovered", 1: "nonhospitalized", 2: "hospitalized", 3: "deceased"}, columns = {0:"recovered", 1: "nonhospitalized", 2: "hospitalized", 3: "deceased"})

,recovered,nonhospitalized,hospitalized,deceased
recovered,5888,376,11440,7
nonhospitalized,92,29942,10,5
hospitalized,999,5,23881,3
deceased,87,79,683,30


In [ ]:
clf = AdaBoostClassifier(n_estimators=10, random_state=0)
clf.fit(X_train, y_train)
pickle.dump( clf, open( "../models/AdaBoost10.pkl", "wb" ) )

In [ ]:
trueAdaBoost = pickle.load( open( '../models/AdaBoost10.pkl', "rb" ) )
adaBoostprediction = list(trueAdaBoost.predict(X_test))
adaConfusionMatrix = confusion_matrix(y_test, adaBoostprediction, labels=["recovered","nonhospitalized","hospitalized","deceased"])

In [ ]:
pd.DataFrame(adaConfusionMatrix).rename(index = {0:"recovered", 1: "nonhospitalized", 2: "hospitalized", 3: "deceased"}, columns = {0:"recovered", 1: "nonhospitalized", 2: "hospitalized", 3: "deceased"})

,recovered,nonhospitalized,hospitalized,deceased
recovered,843,782,16086,0
nonhospitalized,3,30046,0,0
hospitalized,531,38,24319,0
deceased,23,140,716,0


In [ ]:
clf = RandomForestClassifier(n_estimators=25, random_state=0) 
clf.fit(X_train, y_train)
pickle.dump( clf, open( "../models/RandomForest25.pkl", "wb" ) )

In [ ]:
trueRandomForest = pickle.load( open( '../models/RandomForest25.pkl', "rb" ) )
randomForestprediction = list(trueRandomForest.predict(X_test))
randomForestConfusionMatrix = confusion_matrix(y_test, randomForestprediction, labels=["recovered","nonhospitalized","hospitalized","deceased"])

In [ ]:
pd.DataFrame(randomForestConfusionMatrix).rename(index = {0:"recovered", 1: "nonhospitalized", 2: "hospitalized", 3: "deceased"}, columns = {0:"recovered", 1: "nonhospitalized", 2: "hospitalized", 3: "deceased"})

,recovered,nonhospitalized,hospitalized,deceased
recovered,10562,174,6955,20
nonhospitalized,182,29833,13,21
hospitalized,3324,4,21548,12
deceased,256,55,528,40


In [102]:
%%time
OverRF = pickle.load( open( '/content/drive/MyDrive/DoR/rf10overSample', "rb" ) )

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


CPU times: user 9.77 ms, sys: 2.96 ms, total: 12.7 ms
Wall time: 729 ms


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomizedSearchCV from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [115]:
bestest = OverRF.best_estimator_

In [119]:
%%time
randomForestprediction = list(bestest.predict(X_test))
randomForestConfusionMatrix = confusion_matrix(y_test, randomForestprediction, labels=["recovered","nonhospitalized","hospitalized","deceased"])

CPU times: user 740 ms, sys: 9.1 ms, total: 749 ms
Wall time: 751 ms


In [120]:
randomForestConfusionMatrix

array([[ 6964,    82,  7566,  3102],
       [  276, 29161,    11,   526],
       [ 4380,     1, 16685,  3842],
       [  101,    17,   407,   406]])